In [1]:
import math, random
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cv
import pandas as pd

In [2]:
def bootstrap_data(x_list, samples, N, plot=False, printing=False):
    """
    Bootstrap sample data to find confidence intervals of
    stationary distribution values.

    Inputs:
    x_list: states to calculate stationary estimates for
    samples: list of samples from stationary distribution (counts of transcripts)
    N: number of bootstrap samples
    plot: whether to plot histograms of bootstrap estimates with CI lines
    printing: whether to print CI's

    Output:
    intervals: list of confidence intervals for stationary distribution values
        of states in x_list
    """

    # simulate N bootstrap samples: estimates p(x) for each, and for each x
    n = len(samples)
    estimates = [[] for x in x_list]
    for i in range(N):
        sample = random.choices(samples,k = n)
        for i, x in enumerate(x_list):
            estimates[i].append(sample.count(x) / n)

    # create confidence intervals (95%) via 2.5%, 97.5% quantiles for each x
    intervals = [np.quantile(est,[0.025,0.975]) for est in estimates]
    
    # plot histograms and CI
    for i, x in enumerate(x_list):
        if printing:
            print(f"95% CI for p({x}) is: ({intervals[i][0]}, {intervals[i][1]})")
        if plot:
            plt.hist(estimates[i])
            plt.title(f"Hist of p({x})")
            plt.axvline(intervals[i][0], color="red")
            plt.axvline(intervals[i][1], color="red")
            plt.show()

    # return CIs
    return intervals

In [3]:
def bursty_LP(N, intervals, k0=None, k1=None, M=None):
    """
    Solve bursty LP system to get interval bounds on parameters.

    Inputs:
    N: number of rows of Q / number of equations used
    M: maximum burst size to consider (>= 1)
    intervals: confidence intervals for at least N + 1 states

    Outputs:
    solutions: dict with keys variables, values their solution intervals
    """
    # if max burst not specified: set to the largest burst that appears in Qp = 0 constraint
    if not M:
        M = N - 1

    # create Qr matrices: N rows requires N + 1 columns to include all terms (so (N, N + 1) size)
    # Q1: degradation
    Q1 = (np.diag([-x for x in range(0,N+1)],0) + np.diag([x for x in range(1,N+1)],1))[:-1, :]
    # Q00, Q01, ... : bursting
    Q00 = np.diag([-1 for x in range(0, N + 1)])[:-1, :]
    def Q0(m):
        """m = {1, 2, ...}"""
        return np.diag([1 for x in range(0, N + 1 - m)], -m)[:-1, :]

    # bounds from CI
    pl = [intr[0] for intr in intervals]
    pu = [intr[1] for intr in intervals]

    # define bounds: truncate to N + 1 values as using up to Nth equation
    pl = np.array(pl)[:N + 1]
    pu = np.array(pu)[:N + 1]

    # Construct the problem

    # Variables: specify k1 or k0 manually
    if not k0:
        k0 = cv.Variable()
    if not k1:
        k1 = cv.Variable(1)
    mu = cv.Variable(M)
    z0 = cv.Variable(N + 1)
    z1 = cv.Variable(N + 1)
    y = cv.Variable((N + 1, M))

    # constraints
    constraints = [
        Q1 @ z1 + Q00 @ z0 + sum([Q0(m + 1) @ y[:, m] for m in range(0, M)]) == 0,
        k0 >= 0, k1 >= 0, mu >= 0, z0 >= 0, z1 >= 0, y >= 0,
        k0 * pl <= z0, z0 <= k0 * pu,
        k1 * pl <= z1, z1 <= k1 * pu,
        sum([y[:, m] for m in range(0, M)]) <= z0,
        sum([mu[m] for m in range(0, M)]) <= k0
    ]
    for m in range(0, M):
        constraints +=  [
            mu[m] * pl <= y[:, m], y[:, m] <= mu[m] * pu
        ]

    # solution interval dict
    solutions = {}

    # solver function
    def solver(dict_name, var, solutions):
        # create inteval
        solutions[dict_name] = []
        # min and max objectives
        objective_min = cv.Minimize(var)
        objective_max = cv.Maximize(var)
        # min and max problems
        prob_min = cv.Problem(objective_min, constraints)
        prob_max = cv.Problem(objective_max, constraints)   
        # solve min: add to solution
        result_min = prob_min.solve()#verbose=True)
        try:
            solutions[dict_name].append(var.value.item())
        except:
            solutions[dict_name].append(None)
        # solve max: add to solution
        results_max = prob_max.solve()#verbose=True)
        try:
            solutions[dict_name].append(var.value.item())
        except:
            solutions[dict_name].append(None)
        # add statuses
        solutions[dict_name].append(prob_min.status)
        solutions[dict_name].append(prob_max.status)

    # solve for k0
    if type(k0) == int:
        solutions['k0'] = k0
    else:
        solver('k0', k0, solutions)

    # solve for k1
    if type(k1) == int:
        solutions['k1'] = k1
    else:
        solver('k1', k1, solutions)

    # solve for mu_1, mu_2, ..., mu_M
    for m in range(0, M):
        solver(f'mu_{m + 1}', mu[m], solutions)

    return solutions

In [17]:
def bound_sample_refined(sample, threshold=5, skip=1, n=1000, N=None, 
                         plot_sample=True, print_bounds=False, 
                         print_status=True, plot_solution=True):
    """
    Given transcript samples from gene, bootstrap and solve LP to bound parameters

    sample: list of up to ~200 counts of transcripts of gene in cells
    threshold: minimum number of occurances in sample needed to include state x
            set to zero to use up to max state available
    skip: number of equations to drop after an infeasible result
    n: number of bootstrap samples
    N: option to manually specify number of equations used
    settings:
    plot_sample: toggle histogram of sample
    print_bounds: toggle printing CI bounds
    print_status: toggle printing feasible/infeasible reports
    plot_solution: toggle plotting solution bounds on distribution
    """

    # find max burst size in sample
    x_max = int(np.nanmax(sample))

    # compute all possible bounds: p(0), ... , p(x_max)
    intervals = bootstrap_data([x for x in range(x_max + 1)], sample, n, printing=print_bounds)

    # dict of states and occurances in sample
    counts = sample.value_counts().to_dict()

    # find max state with more than threshold occurances
    x_max_thresh = x_max
    # look at all states: decreasing from max
    for x in range(x_max, -1, -1):
        if x in counts:
            # check for first state with > threshold occurance
            if counts[x] > threshold:
                # record state
                x_max_thresh = x
                break

    # edge case: need at least bounds up to p(2) to estimate pi_1
    if x_max_thresh < 2:
        print("Edge case")
        x_max_thresh = 2

    # for each state up to x_max_thesh:
    # above threshold # observations => use CI
    # below => use [0,1] bounds
    # track [0,1] bounded states
    non_bounds = []
    for x in range(x_max_thresh + 1):
        if x in counts:
            # below
            if counts[x] < threshold:
                # [0,1] bounds
                intervals[x] = np.array([0.0, 1.0])
                non_bounds.append(x)
        # if not in count: 0 occurances, below threshold (unless = 0)
        elif threshold > 0:
            intervals[x] = np.array([0.0, 1.0])
            non_bounds.append(x)
     
    # plot sample hist and threshold
    if plot_sample:
        plt.hist(sample, bins=x_max);
        plt.title("Histogram of transcript counts for given gene")
        plt.axhline(threshold, 0, 1, color="orange", label=f"threshold {threshold}")
        plt.axvline(x_max, color="red", label=f"Max state {x_max}")
        plt.axvline(x_max_thresh, color="red", label=f"Max threshold state {x_max_thresh}")
        for x in non_bounds:
            plt.axvline(x, color="green")
        plt.legend()
        plt.show()

    # get bounds on p(0), ..., p(x_max_thresh)
    # can use UP TO equation N = x_max, as involves up to p(x_max_thresh)
    # equation N involves pi's up to pi_(N-1), so M = N - 1
    if not N:
        N = x_max_thresh
        M = N - 1
    
    # solve LP using data
    solutions_dist = bursty_LP(N, intervals, k0=1)

    """
    # check if infeasible
    while solutions_dist['mu_1'][2] == 'infeasible':
        if print_status: print(f"N = {N} infeasible")
        # try again with smaller N (drop 'skip' # of equations)
        N -= skip
        M -= skip
        solutions_dist = bursty_LP(N, intervals, k0=1)
        # stop if too few equations
        if N <= skip + 2:
            break
    """
    
    # check if infeasible
    while solutions_dist['mu_1'][2] == 'infeasible':
        if print_status: print(f"N = {N} infeasible")
        # stop if too few equations (cannot reduce any further leads to M = 0)
        if N <= skip + 1:
            break
        # try again with smaller N (drop 'skip' # of equations)
        N -= skip
        M -= skip
        solutions_dist = bursty_LP(N, intervals, k0=1)

    if print_status: print(f"N = {N} feasible:")

    # plot
    if plot_solution:
        # extract distribution bounds
        labels = [f'mu_{m}' for m in range(1, M + 1)]
        lower = [solutions_dist[var][0] for var in labels]
        upper = [solutions_dist[var][1] for var in labels]
        plt.plot(labels, upper, label = "Upper bound")
        plt.plot(labels, lower, label = "Lower bound")
        plt.title("LP Bounds on birth distribution")
        plt.ylabel("Probability")
        plt.xlabel("Birth distribution")
        plt.legend()
        plt.show()

    # return bounds
    return solutions_dist

# First Dataset

In [22]:
# read in data
data = pd.read_csv("SS3_c57_UMIs_concat_cleaned.csv", index_col="Unnamed: 0")
# record
output = pd.DataFrame()
# take samples
for i in range(data.shape[0]):
    sample = data.iloc[i]
    # solve
    solution_refined = bound_sample_refined(sample, threshold=3, print_bounds=False, plot_sample=False, print_status=False, plot_solution=False)
    # record
    df_solution_refined = pd.DataFrame([solution_refined])
    output = pd.concat([output, df_solution_refined], ignore_index=True)
    if (i % 100) == 0:
        print(f"{i}th entry analysed")

0th entry analysed
100th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


200th entry analysed
Edge case


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


300th entry analysed
400th entry analysed
Edge case


...

3400th entry analysed
3500th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


3600th entry analysed
3700th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


3800th entry analysed


In [23]:
output.head()

,k0,k1,mu_1,mu_2,mu_3,mu_4,mu_5,mu_6,mu_7,mu_8,...,mu_278,mu_279,mu_280,mu_281,mu_282,mu_283,mu_284,mu_285,mu_286,mu_287
0,1,"[0.7931034482720015, 2.0578571428580195, optim...","[-2.8743155645824184e-14, 1.0000000000016618, ...","[-2.653118833851089e-14, 1.000000000002961, op...","[-2.277233585339472e-14, 1.00000000000185, opt...","[-2.0845291320012502e-14, 1.0000000000013007, ...","[-1.78409258333694e-14, 1.000000000000075, opt...","[-1.6684337924484522e-14, 0.6713085690214184, ...","[-1.8331054566664728e-14, 0.6713085690084142, ...","[-2.080683345818881e-14, 0.6713085690264122, o...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"[0.9687499999987329, 2.175000000027888, optima...","[-4.4302975334047706e-13, 0.7822580645270424, ...","[-5.764389656699711e-13, 1.0000000000025557, o...","[-1.2240675092646862e-14, 1.0000000000007392, ...","[-1.6441858083192303e-14, 1.0000000000003637, ...","[-2.1362929482612252e-14, 1.0000000000002727, ...","[-2.4189833960325182e-14, 1.0000000000006157, ...","[-2.551215532510408e-14, 1.0000000000062719, o...","[-2.642644212632816e-14, 1.000000000002613, op...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,"[0.14285714285622997, 1.9583333333413973, opti...","[-1.5826038110619332e-15, 1.0000000000001716, ...","[-1.28875407477881e-15, 1.0000000000032607, op...","[-1.5144922004226716e-15, 1.0000000000029117, ...","[-1.733783730464011e-15, 1.0000000000029936, o...","[-1.6062043160786174e-15, 1.0000000000024472, ...","[-1.3349158541399215e-15, 1.0000000000019278, ...","[-1.2279648171697277e-15, 1.0000000000017242, ...","[-1.1330565476099194e-15, 1.0000000000019413, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,"[None, None, infeasible, infeasible]","[None, None, infeasible, infeasible]","[None, None, infeasible, infeasible]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,"[1.232558139526332, 2.541666666669051, optimal...","[-6.177734572025108e-14, 0.6950416849532361, o...","[-8.260653416551917e-14, 1.0000000000059892, o...","[-1.240861725419763e-13, 1.000000000004966, op...","[-1.2663218123212162e-13, 1.0000000000052542, ...","[-1.1012909576197698e-13, 1.0000000000000797, ...","[-7.009955766795354e-14, 0.9120370370404516, o...","[-7.342345014315811e-14, 0.9120370370448645, o...","[-7.164945570328284e-14, 0.7490842490865798, o...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
output.to_csv("SS3_c57_UMIs_concat_bounds.csv")

In [27]:
output.tail()

,k0,k1,mu_1,mu_2,mu_3,mu_4,mu_5,mu_6,mu_7,mu_8,...,mu_278,mu_279,mu_280,mu_281,mu_282,mu_283,mu_284,mu_285,mu_286,mu_287
3870,1,"[2.6666666666571026, 5.666666666747567, optima...","[-2.884255794402093e-14, 0.6743119266068951, o...","[-2.8831642363849657e-14, 0.7596153846184132, ...","[-2.809850642205567e-14, 0.8977272727442397, o...","[-1.2829940340522955e-12, 1.0000000000011613, ...","[-1.3924803173292863e-12, 1.0000000000013547, ...","[-1.4531920921445519e-12, 1.0000000000028832, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3871,1,"[0.7599999999968455, 2.1354166666713827, optim...","[-1.480982350440163e-14, 0.9999999999995, opti...","[-7.555682168825384e-15, 0.9999999999996556, o...","[-6.423426234121325e-15, 0.9999999999997636, o...","[-6.450431799737919e-15, 0.9999999999998844, o...","[-2.1783529679504483e-14, 0.9999999999997479, ...","[-2.0060587000566178e-14, 0.9999999999997307, ...","[-1.8404233557144176e-14, 0.9999999999997038, ...","[-1.5573911047251578e-14, 0.9999999999995901, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3872,1,"[0.11111111110831104, 2.015151515155253, optim...","[-1.8025471209700867e-14, 1.0000000000000229, ...","[-1.7763937990672647e-14, 1.0000000000000207, ...","[-1.7439093986237613e-14, 1.0000000000000215, ...","[-1.7354553479470782e-14, 1.000000000000018, o...","[-1.6922491395481173e-14, 1.0000000000000164, ...","[-1.670076699005672e-14, 1.0000000000000158, o...","[-1.6636964305135386e-14, 1.0000000000010993, ...","[-1.6422394203002877e-14, 1.0000000000000118, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3873,1,"[1.3442622950740508, 2.8947368421080735, optim...","[-4.829363718004907e-12, 1.000000000002107, op...","[-5.184088499537267e-14, 1.0000000000049811, o...","[-3.5975318814833453e-12, 1.0000000000008844, ...","[-3.1494946510791973e-12, 0.7251109701970964, ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3874,1,"[0.8832074375354428, 2.6256250000010337, optim...","[-1.8394171695011924e-13, 1.0000000000022413, ...","[-2.679531971029452e-13, 1.0000000000006575, o...","[-2.4692850612544806e-13, 1.0000000000005187, ...","[-2.435036248872093e-13, 1.0000000000005824, o...","[-2.4110713054228143e-13, 1.0000000000005131, ...","[-2.425424394050436e-13, 1.0000000000004758, o...","[-2.7730082923657637e-13, 1.0000000000005977, ...","[-2.7026822282953346e-13, 1.0000000000007745, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Second dataset: G1

In [19]:
# read in data
data = pd.read_csv("Data/SS3_c57_UMIs_concat_G1_cleaned.csv", index_col="Unnamed: 0")
# record
output = pd.DataFrame()
# take samples
for i in range(data.shape[0]):
    sample = data.iloc[i]
    # solve
    try:
        solution_refined = bound_sample_refined(sample, threshold=2, print_bounds=False, plot_sample=False, print_status=False, plot_solution=False)
    except:
        print("Error")
        solution_refined = None
    # record
    df_solution_refined = pd.DataFrame([solution_refined])
    output = pd.concat([output, df_solution_refined], ignore_index=True)
    if (i % 100) == 0:
        print(f"{i}th entry analysed")

0th entry analysed
100th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


200th entry analysed
Edge case


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


300th entry analysed
400th entry analysed


...

3400th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


3500th entry analysed
3600th entry analysed


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


3700th entry analysed
3800th entry analysed


In [20]:
output.to_csv("Data/SS3_c57_UMIs_concat_G1_bounds_full.csv")